In [ ]:
# pick the 10 most frequent countries
# for each country plot a line of the pay gap by connecting
# the pay gaps in the different age bins
# x-axis is age bins, y-axis is pay gap
# resulting in 10 lines with country names as labels